In [1]:
from dotenv import load_dotenv
load_dotenv()

import os 
os.environ["OPENAI_API_KEY"]=os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [2]:
from langchain_core.messages import AIMessage, HumanMessage
from pprint import pprint

messages=[AIMessage(content=f"please tell me how can i help",name="LLMModel")]
messages.append(HumanMessage(content=f"I want to learn coding",name="Sai"))
messages.append(AIMessage(content=f"Which programming language you want to learn",name="LLMModel"))
messages.append(HumanMessage(content=f"I want to learn Python Programming Language",name="Sai"))

for message in messages:
    message.pretty_print()

================================== Ai Message ==================================
Name: LLMModel

please tell me how can i help
================================ Human Message =================================
Name: Sai

I want to learn coding
================================== Ai Message ==================================
Name: LLMModel

Which programming language you want to learn
================================ Human Message =================================
Name: Sai

I want to learn Python Programming Language


In [3]:
from langchain_openai import ChatOpenAI
llm=ChatOpenAI(model="gpt-4o")
result=llm.invoke(messages)

In [4]:
result.response_metadata

{'token_usage': {'completion_tokens': 525,
  'prompt_tokens': 57,
  'total_tokens': 582,
  'completion_tokens_details': {'accepted_prediction_tokens': 0,
   'audio_tokens': 0,
   'reasoning_tokens': 0,
   'rejected_prediction_tokens': 0},
  'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}},
 'model_name': 'gpt-4o-2024-08-06',
 'system_fingerprint': 'fp_a288987b44',
 'id': 'chatcmpl-BnSTYVrlw7uSEfLomDT3ypiCJp7Gx',
 'service_tier': 'default',
 'finish_reason': 'stop',
 'logprobs': None}

In [5]:
def add(a: int,b: int)-> int:
    """ Add a and b

    Args:
        a (int): _description_
        b (int): _description_

    Returns:
        int
    """
    return a+b

In [6]:
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x00000255FF94FFE0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x00000255FFBB41D0>, root_client=<openai.OpenAI object at 0x00000255FE424EC0>, root_async_client=<openai.AsyncOpenAI object at 0x00000255FF790F20>, model_name='gpt-4o', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [8]:
llm_with_tools = llm.bind_tools([add])
tool_call=llm_with_tools.invoke([HumanMessage(content=f"What is 2 plus 2",name="Sai")])

In [9]:
tool_call.tool_calls

[{'name': 'add',
  'args': {'a': 2, 'b': 2},
  'id': 'call_5MIwaYnGw7ZmFGOHbUxbwnNu',
  'type': 'tool_call'}]

In [10]:
# using messages as state
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage

class state(TypedDict):
    message:list[AnyMessage]

Now, we have a minor problem!
As we discussed, each node will return a new value for our state key messages.
But, this new value will override the prior messages value.
As our graph runs, we want to append messages to our messages state key.
We can use reducer functions to address this.
Reducers allow us to specify how state updates are performed.
If no reducer function is specified, then it is assumed that updates to the key should override it as we saw before.
But, to append messages, we can use the pre-built add_messages reducer.
This ensures that any messages are appended to the existing list of messages.
We simply need to annotate our messages key with the add_messages reducer function as metadata.

In [11]:
from langgraph.graph.message import add_messages
from typing import Annotated

class State(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]

In [12]:
initial_messages=[AIMessage(content=f"please tell me how can i help",name="LLMModel")]
initial_messages.append(HumanMessage(content=f"I want to learn coding",name="Sai"))
initial_messages

[AIMessage(content='please tell me how can i help', additional_kwargs={}, response_metadata={}, name='LLMModel'),
 HumanMessage(content='I want to learn coding', additional_kwargs={}, response_metadata={}, name='Sai')]

In [13]:
ai_message=AIMessage(content=f"Which programming language you want to learn",name="LLMModel")
ai_message

AIMessage(content='Which programming language you want to learn', additional_kwargs={}, response_metadata={}, name='LLMModel')

In [15]:
# Reducers add_messages is to append instead of override.
add_messages(initial_messages,ai_message)

[AIMessage(content='please tell me how can i help', additional_kwargs={}, response_metadata={}, name='LLMModel', id='bdc65dbb-9f90-47b1-aaca-a458c294971f'),
 HumanMessage(content='I want to learn coding', additional_kwargs={}, response_metadata={}, name='Sai', id='9971f83f-c354-4896-ad92-752620f54d6f'),
 AIMessage(content='Which programming language you want to learn', additional_kwargs={}, response_metadata={}, name='LLMModel', id='acc50676-9b14-42d6-8c11-39ce99982d3b')]